In [4]:
import os
import numpy as np
import cv2
import time
import json
#from pyimagesearch.shapedetector import ShapeDetector
import argparse
import imutils
import re

In [12]:
from hdfs import Config
client = Config().get_client('dev')
files = client.list('/spark/191125_241563_1')

In [54]:
dir(client)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__registry__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_append',
 '_create',
 '_delete',
 '_get_acl_status',
 '_get_content_summary',
 '_get_file_checksum',
 '_get_file_status',
 '_get_home_directory',
 '_get_trash_root',
 '_list_status',
 '_lock',
 '_mkdirs',
 '_modify_acl_entries',
 '_open',
 '_proxy',
 '_rename',
 '_request',
 '_session',
 '_set_acl',
 '_set_owner',
 '_set_permission',
 '_set_replication',
 '_set_times',
 '_timeout',
 '_urls',
 'acl_status',
 'checksum',
 'content',
 'delete',
 'download',
 'from_options',
 'list',
 'makedirs',
 'parts',
 'read',
 'rename',
 'resolve',
 'root',
 'set_acl',
 'set_owner',
 'set_permission',
 'set_replicatio

In [13]:
files

['frame0.jpg',
 'frame1.jpg',
 'frame10.jpg',
 'frame100.jpg',
 'frame101.jpg',
 'frame102.jpg',
 'frame103.jpg',
 'frame104.jpg',
 'frame105.jpg',
 'frame106.jpg',
 'frame107.jpg',
 'frame108.jpg',
 'frame109.jpg',
 'frame11.jpg',
 'frame110.jpg',
 'frame111.jpg',
 'frame112.jpg',
 'frame113.jpg',
 'frame114.jpg',
 'frame115.jpg',
 'frame116.jpg',
 'frame117.jpg',
 'frame118.jpg',
 'frame119.jpg',
 'frame12.jpg',
 'frame120.jpg',
 'frame121.jpg',
 'frame122.jpg',
 'frame123.jpg',
 'frame124.jpg',
 'frame125.jpg',
 'frame126.jpg',
 'frame127.jpg',
 'frame128.jpg',
 'frame129.jpg',
 'frame13.jpg',
 'frame130.jpg',
 'frame131.jpg',
 'frame132.jpg',
 'frame133.jpg',
 'frame134.jpg',
 'frame135.jpg',
 'frame136.jpg',
 'frame137.jpg',
 'frame138.jpg',
 'frame139.jpg',
 'frame14.jpg',
 'frame140.jpg',
 'frame141.jpg',
 'frame142.jpg',
 'frame143.jpg',
 'frame144.jpg',
 'frame145.jpg',
 'frame146.jpg',
 'frame147.jpg',
 'frame148.jpg',
 'frame149.jpg',
 'frame15.jpg',
 'frame150.jpg',
 'frame

In [16]:
# Frames sorting
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [35]:
# Ball Detection in a frame from .mp4
def detect_yellow_mp4(image):
    image = image[50:410, 0:640]
    original = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower = np.array([20, 30, 140], dtype="uint8")
    upper = np.array([45, 255, 255], dtype="uint8")
    mask = cv2.inRange(image, lower, upper)
    res = []
    x = 0
    y = 0
    k = "la balle n est pas sur le terrain"
    cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
     
    for c in cnts:
        approx = cv2.approxPolyDP(c,0.001*cv2.arcLength(c,True),True)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        if len(approx) > 20 :
            #cv2.drawContours(original, [approx],0,(35,255,12),2)
            x,y,w,h = cv2.boundingRect(c)
            if len(approx) > 60 :
                cv2.rectangle(original, (x, y), (x + w, y + h), (36,255,12), 2)
                k = "La balle est en arrêt ou sous le pied d'un joueur"                
            else :
                cv2.rectangle(original, (x, y), (x + w, y + h), (36,255,12), 2)
                k = "la balle est en jeu"
    cv2.imshow('mask', mask)
    cv2.imshow('original', original)
    cv2.waitKey(10)
    cv2.destroyAllWindows()
    res.append(k)
    res.append(x)
    res.append(y)
    return res
    
# Ball Detection in a frame from .mkv
def detect_yellow_mkv(image):
    image = image[0:450, 0:640]
    original = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower = np.array([20, 50, 100], dtype="uint8")
    upper = np.array([45, 255, 255], dtype="uint8")
    mask = cv2.inRange(image, lower, upper)
    res = []
    x = 0
    y = 0
    k = "la balle n est pas sur le terrain"
    cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
     
    for c in cnts:
        approx = cv2.approxPolyDP(c,0.001*cv2.arcLength(c,True),True)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        if len(approx) > 5 :
            #cv2.drawContours(original, [approx],0,(35,255,12),2)
            x,y,w,h = cv2.boundingRect(c)
            if len(approx) > 30 : 
                cv2.rectangle(original, (x, y), (x + w, y + h), (36,255,12), 2)
                k = "La balle est sous le pied d'un joueur"                
            else :
                cv2.rectangle(original, (x, y), (x + w, y + h), (36,255,12), 2)
                k = "la balle est en jeu"
    cv2.imshow('mask', mask)
    cv2.imshow('original', original)
    cv2.waitKey(10)
    cv2.destroyAllWindows()
    res.append(k)
    res.append(x)
    res.append(y)
    return res

#Detect a ball : returns a dict of the frame and the state of the ball
def ball_detect(frame): 
    rslt = {}
    rslt["frame"] = frame
    if 'mp4' in  frame :
        im = cv2.imread(frame,1)
        rslt["Ball state"] = detect_yellow_mp4(im)[0]
        rslt["Ball position"] = (detect_yellow_mp4(im)[1], detect_yellow_mp4(im)[2])

    elif 'mkv' in frame :
        im = cv2.imread(frame,1)
        rslt["Ball state"] = detect_yellow_mkv(im)[0]
        rslt["Ball position"] = (detect_yellow_mkv(im)[1], detect_yellow_mkv(im)[2])
    return rslt

#Creates a .json with the results of all the path's frames
def result_ball_detect(path):   
    with open('results2.json', 'w') as outfile:
        for element in sorted(os.listdir(path), key=numericalSort):
             if element.endswith('.jpg') :
                        json.dump(ball_detect(element), outfile, indent=2)

In [51]:
def result_ball_detect_spark(path):   
    with client.write('result3.json', encoding = 'utf-8') as writer:
        files = client.walk(path)
        for element in files:
            for x in element[0]:
                 if x.endswith('.jpg') :
                        json.dump(ball_detect(element), writer, indent=2)

In [52]:
cv2.imread('/spark/191125_241563_1/frame0.jpg')

In [46]:
list(client.walk("/spark/191125_241563_1"))

[('/spark/191125_241563_1',
  [],
  ['frame0.jpg',
   'frame1.jpg',
   'frame10.jpg',
   'frame100.jpg',
   'frame101.jpg',
   'frame102.jpg',
   'frame103.jpg',
   'frame104.jpg',
   'frame105.jpg',
   'frame106.jpg',
   'frame107.jpg',
   'frame108.jpg',
   'frame109.jpg',
   'frame11.jpg',
   'frame110.jpg',
   'frame111.jpg',
   'frame112.jpg',
   'frame113.jpg',
   'frame114.jpg',
   'frame115.jpg',
   'frame116.jpg',
   'frame117.jpg',
   'frame118.jpg',
   'frame119.jpg',
   'frame12.jpg',
   'frame120.jpg',
   'frame121.jpg',
   'frame122.jpg',
   'frame123.jpg',
   'frame124.jpg',
   'frame125.jpg',
   'frame126.jpg',
   'frame127.jpg',
   'frame128.jpg',
   'frame129.jpg',
   'frame13.jpg',
   'frame130.jpg',
   'frame131.jpg',
   'frame132.jpg',
   'frame133.jpg',
   'frame134.jpg',
   'frame135.jpg',
   'frame136.jpg',
   'frame137.jpg',
   'frame138.jpg',
   'frame139.jpg',
   'frame14.jpg',
   'frame140.jpg',
   'frame141.jpg',
   'frame142.jpg',
   'frame143.jpg',
   'fr